In [1]:
import numpy as np
import torch
import glob
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
class CustomDataset(Dataset):
    def __init__(self):
        self.imgs_path = "/content/drive/MyDrive/lang/language_data/"
        file_list = glob.glob(self.imgs_path + "*")
        self.data = []
        for class_path in file_list:
          class_name = class_path.split("/")[-1]
          for img_path in glob.glob(class_path + "/*.png"):
            print(class_name)
            print(img_path)
            self.data.append([img_path, class_name])
        self.class_map = {"cn" : 0, "el": 1, "en": 2,  "il": 3,  "pl": 4,  "ru": 5 }
        self.img_dim = (256, 256)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = cv2.imread(img_path)
        img = cv2.resize(img, self.img_dim)
        class_id = self.class_map[class_name]
        img_tensor = torch.from_numpy(img)
        img_tensor = img_tensor.permute(2, 0, 1)
        class_id = torch.tensor([class_id])
        return img_tensor.float(), class_id

In [ ]:
dataset = CustomDataset()
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
dataset.__len__()

In [5]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [6]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(32768, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        
        return out

In [8]:
num_classes = 6
num_epochs = 10
batch_size = 4
learning_rate = 0.005
model = None
model = VGG16(num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  


# Train the model
total_step = len(train_loader)


In [9]:

total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        labels = labels[:,0]
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            labels = labels[:,0]
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total)) 

from google.colab import files
torch.save(model.state_dict(), 'model_vgg.pth')
files.download('model_vgg.pth')


Epoch [1/10], Step [76/76], Loss: 11.2767
Accuracy of the network on the 5000 validation images: 56.90515806988353 %
Epoch [2/10], Step [76/76], Loss: 22.8624
Accuracy of the network on the 5000 validation images: 65.3910149750416 %
Epoch [3/10], Step [76/76], Loss: 0.5814
Accuracy of the network on the 5000 validation images: 80.5324459234609 %
Epoch [4/10], Step [76/76], Loss: 0.0147
Accuracy of the network on the 5000 validation images: 88.6855241264559 %
Epoch [5/10], Step [76/76], Loss: 1.6927
Accuracy of the network on the 5000 validation images: 92.67886855241265 %
Epoch [6/10], Step [76/76], Loss: 2.3883
Accuracy of the network on the 5000 validation images: 76.87188019966722 %
Epoch [7/10], Step [76/76], Loss: 1.0380
Accuracy of the network on the 5000 validation images: 93.34442595673877 %
Epoch [8/10], Step [76/76], Loss: 0.5567
Accuracy of the network on the 5000 validation images: 75.04159733777038 %
Epoch [9/10], Step [76/76], Loss: 1.1479
Accuracy of the network on the 5

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
zero_count = 0
one_count = 0
two_count = 0
three_count = 0
four_count = 0
five_count = 0
for image, label in train_dataset:
  if label.item() == 0:
    zero_count +=1
  if label.item() == 1:
    one_count +=1
  if label.item() == 2:
    two_count +=1
  if label.item() == 3:
    three_count +=1
  if label.item() == 4:
    four_count +=1
  if label.item() == 5:
    five_count +=1
print(zero_count)
print(one_count)
print(two_count)
print(three_count)
print(four_count)
print(five_count)


416
396
396
395
404
397


In [12]:
zero_count = []
one_count = []
two_count = []
three_count = []
four_count = []
five_count = []

def counts(t, o):
  if o == 0:
    zero_count.append(1)
  if o == 1:
    one_count.append(1)
  if o == 2:
    two_count.append(1)
  if o == 3:
    three_count.append(1)
  if o == 4:
    four_count.append(1)
  if o == 5:
    five_count.append(1)
for images, labels in train_loader:
  labels = labels[:,0]
  images = images.to(device)
  labels = labels.to(device)
  outputs = model_pre(images)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  for x in range(len(predicted)):
    counts(labels[x].item(), predicted[x].item())
  del images, labels, outputs
print(len(zero_count))
print(len(one_count))
print(len(two_count))
print(len(three_count))
print(len(four_count))
print(len(five_count))



416
400
333
395
472
388


In [11]:
model_pre = None
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_pre = VGG16(6).to(device)
model_pre.load_state_dict(torch.load('model_vgg.pth'))
model_pre.eval()

VGG16(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=

In [53]:
correct = 0
total = 0
for images, labels in test_loader:
  labels = labels[:,0]
  images = images.to(device)
  labels = labels.to(device)
  outputs = model_pre(images)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  del images, labels, outputs



In [33]:
map = {0 : "cn", 1: "el", 2: "en",  3:"il",  4:"pl",  5:"ru"}

In [74]:
print("percentage correct",correct/total)

percentage correct 0.9534109816971714


In [41]:
def most_common_incorrect(t, o):
  if t != o:
    if o == 0:
      as_zero.append(1)
    if o == 1:
      as_one.append(1)
    if o == 2:
      as_two.append(1)
    if o == 3:
      as_three.append(1)
    if o == 4:
      as_four.append(1)
    if o == 5:
      as_five.append(1)


In [71]:
correct = 0
total = 0
as_zero = []
as_one = []
as_two = []
as_three = []
as_four = []
as_five = []
for images, labels in test_loader:
  labels = labels[:,0]
  images = images.to(device)
  labels = labels.to(device)
  outputs = model_pre(images)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  for x in range(len(predicted)):
    if labels[x].item() == 5:
      most_common_incorrect(labels[x].item(), predicted[x].item())
  del images, labels, outputs


In [72]:
print(map[5], "incorrectly classfies as ")
print(map[0], len(as_zero), "times")
print(map[1], len(as_one), "times")
print(map[2], len(as_two), "times")
print(map[3], len(as_three), "times")
print(map[4], len(as_four), "times")
print(map[5], len(as_five), "times")


ru incorrectly classfies as 
cn 0 times
el 0 times
en 0 times
il 0 times
pl 4 times
